In [ ]:
# 05-05. 소프트맥스 회귀로 MNIST 데이터 분류하기 

# 필요한 도구 임포트 
import torch 
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn 
import matplotlib.pyplot as plt
import random 

# GPU/CPU 연산
USE_CUDA = torch.cuda.is_available()    # GPU 를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device('cuda' if USE_CUDA else 'cpu')    # GPU 사용가능하면 사용하고 아니면 CPU 사용 
print('다음 기기로 학습합니다: ', device)

# random seed 고정 
# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda': 
    torch.cuda.manual_seed_all(777)

# hyperparameters
training_epochs = 15
batch_size = 100

# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/', 
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)

mnist_test = dsets.MNIST('MNIST_data/', 
                         train=False, 
                         transform=transforms.ToTensor(), 
                         download=True)

# root: MNIST 데이터를 다운로드받을 경로
# train: True(훈련 데이터 리턴), False(테스트 데이터 리턴)
# transform: 현재 데이터를 파이토치 텐서로 변환
# download: 해당 경로에 MNIST 데이터가 없다면 다운로드 받겠다

# data loader
data_loader = DataLoader(dataset=mnist_train, 
                         batch_size = batch_size,   # 배치 크기는 100
                         shuffle = True, 
                         drop_last = True)

In [ ]:
# MNIST_data image of shape 28 * 28 = 784
linear = nn.Linear(784, 10, bias=True).to(device)

for epoch in range(training_epochs):    # 앞서 training_epochs의 값은 15로 지정함. 
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader: 
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다. 
        X = X.vies(-1, 28 * 28).to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch: ', '%04d' % (epoch + 1), 'cost = ', '{:.9f}'.format(avg_cost))

print('Learning finished')

In [ ]:
# 테스트 데이터를 사용하여 모델을 테스트한다. 
with torch.no_grad():   # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다. 
    X_test = mnist_test.test_data.view(-1, 28 * 28).format().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

    # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다 
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    